In [23]:
import squidpy as sq
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from scipy.sparse import hstack
import json
from scipy.sparse import csr_matrix, csc_matrix
import math


In [2]:
adata = sq.datasets.seqfish()

labels = adata.obs['celltype_mapped_refined'].cat.codes.values
classes = np.unique(labels)
_, counts = np.unique(labels, return_counts=True)

X_with_spatial = csc_matrix(hstack((adata.X, adata.obsm['spatial'])))

In [14]:
def save_results(filename, results):
    with open(f"results/{filename}", 'w') as f:
        for result in results:
            json.dump(result, f)
            f.write("\n")

In [15]:
def get_stats(y_true, y_pred):
    return {
        'accuracy_score': metrics.accuracy_score(y_true, y_pred),
        'balanced_accuracy': metrics.balanced_accuracy_score(y_true, y_pred),
        'f1_score': metrics.f1_score(y_true, y_pred, average='macro', labels=classes, zero_division=0),
        'recall': metrics.recall_score(y_true, y_pred, average='macro', labels=classes, zero_division=0),
        'precision_score': metrics.precision_score(y_true, y_pred, average='macro', labels=classes, zero_division=0),
    }

def average_stats(stats_list):
    keys = stats_list[0].keys()
    n = len(stats_list)
    avg = {}
    for key in keys:
        sum = 0
        for stats in stats_list:
            sum += stats[key]
        avg[key] = sum / n
    return avg

def eval_k_fold(model, x, y, x_rotated=None):
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=1604)
    stats_list = []
    for train_index, test_index in skf.split(x, y):
        model.fit(x[train_index], y[train_index])
        if x_rotated is None:
            pred = model.predict(x[test_index])
        else:
            pred = model.predict(x_rotated[test_index])
        stats_list.append(get_stats(y[test_index], pred))
    avg = average_stats(stats_list)
    print(avg)
    return avg


In [ ]:
svm_results = []

In [16]:
for kernel in ['rbf', 'linear', 'poly']:
    classifier = svm.SVC(kernel=kernel)
    res = eval_k_fold(classifier, adata.X, labels)
    res['description'] = f"No spatial data, kernel: {kernel}"
    svm_results.append(res)

{'accuracy_score': 0.8413679439637413, 'balanced_accuracy': 0.6828352659430069, 'f1_score': 0.6918458992784452, 'recall': 0.6828352659430069, 'precision_score': 0.7290956057127514}
{'accuracy_score': 0.8281829419035848, 'balanced_accuracy': 0.7281286772368624, 'f1_score': 0.7290112621762397, 'recall': 0.7281286772368624, 'precision_score': 0.7323814334540106}
{'accuracy_score': 0.6240214256283477, 'balanced_accuracy': 0.49516211234458024, 'f1_score': 0.5588964861312239, 'recall': 0.49516211234458024, 'precision_score': 0.6986243630057913}


In [17]:
for kernel in ['rbf', 'linear', 'poly']:
    classifier = svm.SVC(kernel=kernel, class_weight='balanced')
    res = eval_k_fold(classifier, adata.X, labels)
    res['description'] = f"No spatial data, balanced, kernel: {kernel}"
    svm_results.append(res)


{'accuracy_score': 0.8203543469303667, 'balanced_accuracy': 0.7471576512078603, 'f1_score': 0.7308501526615956, 'recall': 0.7471576512078603, 'precision_score': 0.7256952092760219}
{'accuracy_score': 0.8203543469303667, 'balanced_accuracy': 0.7471576512078603, 'f1_score': 0.7308501526615956, 'recall': 0.7471576512078603, 'precision_score': 0.7256952092760219}
{'accuracy_score': 0.8203543469303667, 'balanced_accuracy': 0.7471576512078603, 'f1_score': 0.7308501526615956, 'recall': 0.7471576512078603, 'precision_score': 0.7256952092760219}


In [18]:
classifier = svm.SVC(kernel='linear')
res = eval_k_fold(classifier, X_with_spatial, labels)
res['description'] = "With spatial data"
svm_results.append(res)

{'accuracy_score': 0.829264524103832, 'balanced_accuracy': 0.7303053199557742, 'f1_score': 0.7305527279949368, 'recall': 0.7303053199557742, 'precision_score': 0.7339299108801014}


In [19]:
theta = math.pi / 2
R = np.array([[math.cos(theta), -math.sin(theta)],
             [math.sin(theta), math.cos(theta)]])
X_with_spatial_rotated = hstack(
    (X_with_spatial[:, :-2], X_with_spatial[:, -2:] @ csr_matrix(R)))
pred = classifier.predict(X_with_spatial_rotated)
res = eval_k_fold(classifier, X_with_spatial, labels,
                  x_rotated=X_with_spatial_rotated)
res['description'] = "With spatial data, rotated"
svm_results.append(res)


{'accuracy_score': 0.8210754017305315, 'balanced_accuracy': 0.7129611918915871, 'f1_score': 0.7156881576040005, 'recall': 0.7129611918915871, 'precision_score': 0.7259928579521387}


In [20]:
save_results("svm_results", svm_results)

In [21]:
logistic_results = []

classifier = LogisticRegression()
res = eval_k_fold(classifier, adata.X, labels)
res['description'] = "Logistic regression, unbalanced"
logistic_results.append(res)

classifier = LogisticRegression(class_weight='balanced')
res = eval_k_fold(classifier, adata.X, labels)
res['description'] = "Logistic regression, balanced"
logistic_results.append(res)

save_results("logistic_results", logistic_results)


C:\Users\pedro\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\pedro\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

{'accuracy_score': 0.8352389781623404, 'balanced_accuracy': 0.7084716163352285, 'f1_score': 0.7162633856026237, 'recall': 0.7084716163352285, 'precision_score': 0.7276849431845185}


C:\Users\pedro\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\pedro\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

{'accuracy_score': 0.8292130201895344, 'balanced_accuracy': 0.726384720763188, 'f1_score': 0.7233818499279714, 'recall': 0.726384720763188, 'precision_score': 0.7229241824601855}


C:\Users\pedro\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [30]:
mlp_results = []
hidden_layer_sizes = [(128,), (256,), (128,128,), (256,256)]
for hidden_layer_size in hidden_layer_sizes:
    classifier = MLPClassifier(hidden_layer_sizes=hidden_layer_size, alpha=0.0001, early_stopping=False, validation_fraction=0.1, tol=1e-4, learning_rate_init=0.001, n_iter_no_change=10)
    res = eval_k_fold(classifier, adata.X, labels)
    res['description'] = f"MLP classifier, hidden layer size: {hidden_layer_size}"
    print(res)
    mlp_results.append(res)

{'accuracy_score': 0.8489905232797693, 'balanced_accuracy': 0.7332014135424075, 'f1_score': 0.7407069072031857, 'recall': 0.7332014135424075, 'precision_score': 0.7523453085864468}
{'accuracy_score': 0.8489905232797693, 'balanced_accuracy': 0.7332014135424075, 'f1_score': 0.7407069072031857, 'recall': 0.7332014135424075, 'precision_score': 0.7523453085864468, 'description': 'MLP classifier, hidden layer size: (256, 256)'}


In [ ]:
save_results("mlp_results", mlp_results)
